<a href="https://colab.research.google.com/github/whossssssss/NLP/blob/main/NLP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download ru_core_news_sm

2023-09-23 02:22:36.058383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 02:22:37.539800: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm
from spacy.tokens import DocBin
nlp = spacy.load("ru_core_news_sm")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/archive3.zip -d /content

Archive:  /content/drive/MyDrive/archive3.zip
  inflating: /content/Test.csv       
  inflating: /content/Train.csv      
  inflating: /content/Valid.csv      


In [ ]:
df = pd.read_csv('/content/Train.csv')

In [ ]:
df = df.sample(frac=0.5)
df

,text,label
39646,I was also disappointed with this movie. For s...,0
30103,This movie is great. Best acting i have ever s...,1
8262,"Deepa Mehta's ""Fire"" is groundbreaking, bold, ...",1
5025,This is by far one of my favorite of the Ameri...,1
31546,"Bad Actors, bad filming, choppy dialog, shallo...",0
...,...,...
19564,The acting was horrible and they got both of t...,0
2059,This movie was release when I was 15 and I cou...,1
13657,I remember when I first heard about Jack Frost...,0
18934,i saw this before 'bubba ho-tep' at the fantas...,1


In [ ]:
df['label'].value_counts()

0    10086
1     9914
Name: label, dtype: int64

In [ ]:
df = df[df['label'] != 'neautral']

In [ ]:
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]

In [ ]:
df['label'].value_counts()

0    10086
1     9914
Name: label, dtype: int64

In [ ]:
train_data = data[:20000]
valid_data = data[10000:]

In [ ]:
def make_docs(data):
    docs = []
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == '0':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [ ]:
num_texts = 5000
train_docs = make_docs(train_data[:num_texts])


100%|██████████| 5000/5000 [01:50<00:00, 45.25it/s]


In [ ]:
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 5000/5000 [01:48<00:00, 46.09it/s]


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-09-23 02:52:48.631130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output

2023-09-23 02:52:58.588065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       50.00    0.50
  0     200          0.60       50.00    0.50
  0     400          0.02       50.00    0.50
  0     600          0.01       50.00    0.50
  0     800          0.00       50.00    0.50
  0    1000          0.00       50.00    0.50
  0    1200          0.00       50.00    0.50
  0    1400          0.05       50.00    0.50
  0    1600          0.00       50.00    0.50
✔ Saved pipeline to output directory
output/model-last


In [ ]:
import spacy
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
Please enter example input: I am good
{'positive': 0.6038303971290588, 'negative': 0.39616960287094116}
the sentiment is positive
Please enter example input: quit
{'positive': 0.5175004601478577, 'negative': 0.48249953985214233}
the sentiment is positive
